# Bayesian models

In this seminar:

* Make sure all the dependencies are installed and work fine for you
* Look into the lecture example

## Installation
1. Install [Anaconda](https://www.anaconda.com/) or [Mamba](https://github.com/mamba-org/mamba)
    
    * Conda is slow
    * Mamba is fast (I use this one)
    
2. Create an environment for the course

```console
conda env update -f environment.yaml
```

You can find the environment file in the root of the repository

3. Install jupyter kernel for the course

```
conda activate bayes-econ
python -m ipykernel install --user --name bayes-econ
```

In [ ]:
import pymc as pm
import arviz as az
import numpy as np
import pandas as pd
import aesara
import seaborn as sns
import matplotlib.pyplot as plt
import h5py
import statsmodels.api as sm
az.style.use("arviz-white")

## Example 0 - 8 schools

In [ ]:
J = 8
y = np.array([28, 8, -3, 7, -1, 1, 18, 12])
# lower std for better visualizations
sigma = np.array([15, 10, 16, 11, 9, 11, 10, 18]) / 2

We'll use 
* `pm.Normal`
* `pm.HalfCauchy`

## Plotting logp

In [ ]:
...

## The model

$$
\begin{array}{rcl}
            \text{mu} &\sim & \operatorname{N}(0,~5)\\\text{tau} &\sim & \operatorname{C^{+}}(0,~5)\\\text{theta}_i &\sim & \operatorname{N}(\text{mu},~\text{tau})\\\text{obs}_i &\sim & \operatorname{N}(\text{theta},~\sigma_i)
            \end{array}
$$

In [ ]:
with pm.Model(coords=dict(J=range(J))) as schools_centered:
    mu = ...
    tau = ...
    theta = ...
    obs = ...

In [ ]:
pm.model_to_graphviz(schools_centered)

In [ ]:
schools_centered

In [ ]:
with schools_centered:
    trace = pm.sample()

In [ ]:
trace

## Example 1 - Where Do Priors Come From?

In [ ]:
data = pd.read_csv(
    "https://osf.io/vfjud/download", 
    sep="\t",
    names=["id", "QF", "sopt1", "sopt2", "sopt3", "sopt4", "cudit1"], 
    na_values="999"
)

In [ ]:
data

> Their "id" variable seem to have duplicates

In [ ]:
long_data = (
    pd.wide_to_long(data.reset_index(), "sopt", "index", "t")
    .reset_index()
    .dropna()
    .assign(t=lambda df: df.t-1)
)
long_data

## Visualizing data

In [ ]:
data.loc[:, "sopt1":"sopt4"].T.plot(legend=False, color="b", alpha=0.2);
avg = data.loc[:, "sopt1":"sopt4"].mean().plot(color="b", linestyle="--", lw=4, label="avg")
plt.title("SOPT scores")
plt.xlabel("Tests 1-4")
plt.ylabel("Score");

The data is split by "CUDIT" variable which splits heavy and light users

In [ ]:
ax = data.query("cudit1 == 0").loc[:, "sopt1":"sopt4"].T.plot(legend=False, color="blue", alpha=0.2);
data.query("cudit1 == 0").loc[:, "sopt1":"sopt4"].mean().plot(color="blue", linestyle="--", lw=4, label="avg", ax=ax)
data.query("cudit1 == 1").loc[:, "sopt1":"sopt4"].T.plot(legend=False, color="red", alpha=1, ax=ax);
data.query("cudit1 == 1").loc[:, "sopt1":"sopt4"].mean().plot(color="red", linestyle="--", lw=4, label="avg", ax=ax)
plt.title("SOPT scores")
plt.xlabel("Tests 1-4")
plt.ylabel("Score");

## The model

#### Functional form
$$
\text{SOPT}_t = a_1 + a_2 * t + a_3 * t ^ 2
$$

* $a_1$ - intercept
* $a_2$ - linear trend
* $a_3$ - quadratic term, possible decay in growth
* $t$ - $0\dots 3$, periods where measurement was taken

### Priors (by Authors)
$$
\begin{align}
\text{SOPT}_t &\sim \operatorname{Normal}(a_1 + a_2 * t + a_3 * t ^ 2, \sigma)\\
a_1 &\sim \operatorname{Normal}(85, \sqrt{30})\\
a_2 &\sim \operatorname{Normal}(2, \sqrt{7.5})\\
a_3 &\sim \operatorname{Normal}(-0.1, \sqrt{7.5})\\
\sigma &\sim \operatorname{HalfFlat}
\end{align}
$$

> $\sigma$ is the most unclear part of the paper. 
> 
> Authors used MPLUS library whith the following [docs](https://www.rensvandeschoot.com/regression-in-mplus-bayesian/)
> Which advertises $\sigma \sim \operatorname{I-Gamma}(-1, 0)$.
> 
> This is HalfFlat distribution (the most obscure choice)

#### Adding Groups
Groups are added in a very simplistic way

$$
\begin{align}
\text{SOPT}^g_t &\sim \operatorname{Normal}(a^g_1 + a^g_2 * t + a_3 * t ^ 2, \sigma)\\
a^g_1 &\sim \operatorname{Normal}(85, \sqrt{30})\\
a^g_2 &\sim \operatorname{Normal}(2, \sqrt{7.5})\\
a_3 &\sim \operatorname{Normal}(-0.1, \sqrt{7.5})\\
a^0_2 - a^1_2 &\sim \operatorname{Normal}(0, 10)\\
\sigma &\sim \operatorname{HalfFlat}
\end{align}
$$

##### Potential Trick

You can see a non standard interaction
$$
a^0_2 - a^1_2 \sim \operatorname{Normal}(0, 10)
$$
With PyMC this can be incorporated into the model with a Potential.

In [ ]:
long_data.head()

In [ ]:
import itertools
with pm.Model(coords=dict(cudit=["light", "heavy"], period=range(4))) as model:
    # a1 and a2 priors are simple
    # dims="cudit" will notify the model to add extra dims
    a1 = ...
    # coordinates help in visualizations
    a2 = ...
    
    # shared quadratic term
    a3 = ...
    
    # Half Flat sigma (improper prior)
    σ = ...
    # creating a matrix with items (cudit x period)
    # a^g_1 + a^g_2 * t + a_3 * t ^ 2
    sopt = (
        ...
    )
    
    # you can use numpy indexing to get an estimate for each observation obs_{itc}
    
    observed = ...

In [ ]:
with model:
    trace = pm.sample()

## Working with a posterior using xarray

In [ ]:
import xarray as xa

In [ ]:
time = xa.DataArray(np.arange(4), dims="time", name="time")

In [ ]:
sns.boxplot(x="time", y="y", hue="cudit", data=df.reset_index());